In [1]:
from sae_lens import SAE, LanguageModelSAERunnerConfig, SAEConfig
from datasets import load_dataset  
from transformer_lens import HookedTransformer
import torch
import plotly.express as px

/mnt/c/Users/thijm/GitHub/Master_related/ProjectAI_related/OthelloGPT-SAELens/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
token_dataset = load_dataset('taufeeque/othellogpt', split='train')
model = HookedTransformer.from_pretrained('my-own-othello-model')
sae = SAE.load_from_pretrained('othello_sae')

This happend
{'n_layers': 6, 'd_model': 128, 'd_mlp': 512, 'd_head': 64, 'n_heads': 8, 'n_ctx': 59, 'd_vocab': 61, 'act_fn': 'gelu', 'attn_only': False, 'normalization_type': 'LNPre'}
Loaded pretrained model my-own-othello-model into HookedTransformer


In [3]:
batch_tokens = token_dataset[:32]["tokens"]
print(batch_tokens)

[[20, 21, 34, 19, 13, 40, 47, 28, 12, 41, 35, 5, 10, 43, 3, 23, 6, 26, 42, 54, 52, 51, 59, 36, 39, 44, 11, 49, 4, 27, 32, 38, 33, 1, 25, 60, 48, 17, 50, 57, 29, 58, 55, 14, 22, 46, 30, 24, 9, 31, 16, 8, 56, 7, 45, 18, 15, 2, 37, 53], [34, 42, 27, 19, 50, 51, 52, 35, 41, 48, 49, 56, 36, 58, 11, 60, 20, 40, 57, 30, 24, 28, 59, 2, 21, 44, 22, 12, 13, 26, 4, 14, 15, 23, 5, 43, 33, 32, 10, 8, 16, 47, 55, 54, 46, 45, 37, 7, 53, 31, 39, 9, 29, 38, 25, 18, 3, 6, 17, 1], [20, 21, 22, 13, 14, 19, 5, 15, 29, 11, 16, 8, 28, 6, 27, 33, 4, 3, 18, 7, 2, 12, 41, 10, 32, 23, 9, 37, 24, 40, 39, 50, 25, 17, 59, 47, 1, 42, 51, 36, 54, 55, 35, 30, 34, 43, 52, 26, 48, 60, 56, 46, 45, 57, 38, 58, 31, 49, 44, 53], [34, 28, 21, 12, 23, 42, 13, 22, 33, 39, 41, 14, 43, 50, 29, 27, 40, 49, 4, 24, 16, 20, 57, 48, 19, 30, 18, 51, 32, 52, 5, 26, 47, 9, 60, 37, 31, 35, 17, 38, 7, 6, 58, 3, 45, 56, 2, 59, 44, 8, 36, 15, 11, 25, 46, 10, 55, 53, 1, 54], [20, 19, 41, 21, 11, 28, 10, 40, 39, 49, 34, 38, 14, 13, 29, 27, 48

In [4]:
!pip install nbformat>=4.2.0


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
sae.eval()
sae.cuda()
with torch.no_grad():
    # activation store can give us tokens.
    batch_tokens = token_dataset[:32]["tokens"]
    # convert to tensor
    batch_tokens = torch.tensor(batch_tokens)[:, -1]
    _, cache = model.run_with_cache(batch_tokens.cuda(), prepend_bos=False)
    
    
    feature_acts = sae.encode(cache[sae.cfg.hook_name])
    sae_out = sae.decode(feature_acts)
    
    
    l0 = (feature_acts[:, 1:] > 0).float().sum(-1).detach()
    print("average l0", l0.mean().item())
    px.histogram(l0.flatten().cpu().numpy()).show()

average l0 18.096773147583008


In [11]:
from transformer_lens import utils
from functools import partial

# next we want to do a reconstruction test.
def reconstr_hook(activation, hook, sae_out):
    return sae_out


def zero_abl_hook(activation, hook):
    return torch.zeros_like(activation)


print("Orig", model(batch_tokens, return_type="loss").item())
print(
    "reconstr",
    model.run_with_hooks(
        batch_tokens,
        fwd_hooks=[
            (
                sae.cfg.hook_name,
                partial(reconstr_hook, sae_out=sae_out),
            )
        ],
        return_type="loss",
    ).item(),
)
print(
    "Zero",
    model.run_with_hooks(
        batch_tokens,
        return_type="loss",
        fwd_hooks=[(sae.cfg.hook_name, zero_abl_hook)],
    ).item(),
)

Orig 5.2594428062438965
reconstr 5.097574234008789
Zero 5.093095302581787


In [27]:
from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData

test_feature_idx_gpt = list(range(10)) + [14057]
hook_name = sae.cfg.hook_name

batch_tokens = token_dataset[:32]["tokens"]
# convert to tensor
batch_tokens = torch.tensor(batch_tokens)[:, -1]
    
feature_vis_config_gpt = SaeVisConfig(
    hook_point=hook_name,
    features=test_feature_idx_gpt,
    batch_size=2048,
    minibatch_size_tokens=128,
    verbose=True,
)
sae = SAE.load_from_pretrained('othello_sae')
token_dataset = load_dataset('taufeeque/othellogpt', split='train')

sae.eval()

sae_vis_data_gpt = SaeVisData.create(
    encoder=sae.cuda(),
    model=model.cuda(), # type: ignore
    tokens=token_dataset[:32]["tokens"],  # type: ignore
    cfg=feature_vis_config_gpt,
)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
